In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# В первой части показана обработка датасетов и дообучение
# В конце блокнота показаны тесты по метрике ROUGE

Возьмем датасет с новостями Лента.ру, который любезно предоставлен нам в Kaggle. Посмотрим какие в нем уникальные топики.

In [1]:
import pandas as pd

data = pd.read_csv('/kaggle/input/corpus-of-russian-news-articles-from-lenta/lenta-ru-news.csv')

unique_topics = data['topic'].unique()

print(unique_topics)

['Библиотека' 'Россия' 'Мир' 'Экономика' 'Интернет и СМИ' 'Спорт'
 'Культура' 'Из жизни' 'Силовые структуры' 'Наука и техника' 'Бывший СССР'
 nan 'Дом' 'Сочи' 'ЧМ-2014' 'Путешествия' 'Ценности' 'Легпром' 'Бизнес'
 'МедНовости' 'Оружие' '69-я параллель' 'Культпросвет ' 'Крым']


<ipython-input-1-3f0064508d55>:3: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('/kaggle/input/corpus-of-russian-news-articles-from-lenta/lenta-ru-news.csv')


Посмотрим распределение этих топиков.

In [2]:
import pandas as pd

topic_distribution = data['topic'].value_counts()

print(topic_distribution)


topic
Россия               160445
Мир                  136621
Экономика             79528
Спорт                 64413
Культура              53797
Бывший СССР           53402
Наука и техника       53136
Интернет и СМИ        44663
Из жизни              27605
Дом                   21734
Силовые структуры     19596
Ценности               7766
Бизнес                 7399
Путешествия            6408
69-я параллель         1268
Крым                    666
Культпросвет            340
Легпром                 114
Библиотека               65
Оружие                    3
ЧМ-2014                   2
МедНовости                1
Сочи                      1
Name: count, dtype: int64


Теперь сделаем 6 датасетов для нашего исследования: 2 по "науке" (6900 и 15000 статей), 2 по "спорту" (6900 и 15000 статей) и 2 по совокупности "наука" + "спорт" (6900 и 15000). Ниже показан пример с созданием последнего датасета по "науке" на 15000 статей.

In [4]:
import pandas as pd


selected_topic = 'Наука и техника'
number_of_samples = 15000  

filtered_data = data[data['topic'] == selected_topic]

sampled_data = filtered_data.sample(n=min(number_of_samples, len(filtered_data)), random_state=42)

sampled_data.to_csv('/kaggle/working/nauka_15000.csv', index=False) #сохраним, чтобы потом перевести в json и возможно использовать в будущем для других исследований

print(sampled_data)


                                                url  \
361028    https://lenta.ru/news/2011/01/27/approve/   
200220     https://lenta.ru/news/2007/09/06/crater/   
428448     https://lenta.ru/news/2012/07/20/petrel/   
464233  https://lenta.ru/news/2013/05/01/atommovie/   
383435      https://lenta.ru/news/2011/07/27/mango/   
...                                             ...   
559785        https://lenta.ru/news/2015/09/07/iss/   
106736  https://lenta.ru/news/2004/11/25/meteorite/   
157510    https://lenta.ru/news/2006/06/30/micropc/   
309645        https://lenta.ru/news/2009/12/01/cod/   
235979   https://lenta.ru/news/2008/07/04/poseidon/   

                                                    title  \
361028     Евросоюз одобрил многомиллиардную сделку Intel   
200220  На месте падения "Протона" образовалась воронк...   
428448  Соседский запах показался птицам привлекательн...   
464233                    IBM показала анимацию из атомов   
383435  Представлен первый смартфо

In [5]:
sampled_data

,url,title,text,topic,tags,date
361028,https://lenta.ru/news/2011/01/27/approve/,Евросоюз одобрил многомиллиардную сделку Intel,Европейский союз одобрил сделку по покупке кор...,Наука и техника,Все,2011/01/27
200220,https://lenta.ru/news/2007/09/06/crater/,"На месте падения ""Протона"" образовалась воронк...","В момент падения российской ракеты-носителя ""П...",Наука и техника,Все,2007/09/06
428448,https://lenta.ru/news/2012/07/20/petrel/,Соседский запах показался птицам привлекательн...,"Ученые установили, что избегать близкородствен...",Наука и техника,Все,2012/07/20
464233,https://lenta.ru/news/2013/05/01/atommovie/,IBM показала анимацию из атомов,Инженеры IBM Research создали самую маленькую ...,Наука и техника,Наука,2013/05/01
383435,https://lenta.ru/news/2011/07/27/mango/,Представлен первый смартфон на Windows Phone 7...,Компании Toshiba и Fujitsu на мероприятии в То...,Наука и техника,Все,2011/07/27
...,...,...,...,...,...,...
559785,https://lenta.ru/news/2015/09/07/iss/,ЦУП скорректировал орбиту МКС,"Центр управления полетами в понедельник, 7 сен...",Наука и техника,Космос,2015/09/07
106736,https://lenta.ru/news/2004/11/25/meteorite/,Впервые сфотографировано столкновение метеорит...,Фотограф-любитель на севере Австралии впервые ...,Наука и техника,Все,2004/11/25
157510,https://lenta.ru/news/2006/06/30/micropc/,В Израиле создан компьютер размером с кредитну...,Израильская компания Compulab представила супе...,Наука и техника,Все,2006/06/30
309645,https://lenta.ru/news/2009/12/01/cod/,Объявлена дата выхода первой части Call of Dut...,Издательство Activision выпустит первую часть ...,Наука и техника,Все,2009/12/01


Теперь разделим датасеты на обучающую и валидационную выборки и переведем в формат jsonl, необходимый для дообучения. 

In [8]:
data_csv_path1 = "/kaggle/working/nauka_15000.csv"

In [10]:
import pandas as pd
import chardet
import csv

with open(data_csv_path1, 'r', encoding="UTF-8") as f:
  data1 = pd.read_csv(f, encoding = 'utf-8', engine="python")

In [11]:
data1.head()

,url,title,text,topic,tags,date
0,https://lenta.ru/news/2011/01/27/approve/,Евросоюз одобрил многомиллиардную сделку Intel,Европейский союз одобрил сделку по покупке кор...,Наука и техника,Все,2011/01/27
1,https://lenta.ru/news/2007/09/06/crater/,"На месте падения ""Протона"" образовалась воронк...","В момент падения российской ракеты-носителя ""П...",Наука и техника,Все,2007/09/06
2,https://lenta.ru/news/2012/07/20/petrel/,Соседский запах показался птицам привлекательн...,"Ученые установили, что избегать близкородствен...",Наука и техника,Все,2012/07/20
3,https://lenta.ru/news/2013/05/01/atommovie/,IBM показала анимацию из атомов,Инженеры IBM Research создали самую маленькую ...,Наука и техника,Наука,2013/05/01
4,https://lenta.ru/news/2011/07/27/mango/,Представлен первый смартфон на Windows Phone 7...,Компании Toshiba и Fujitsu на мероприятии в То...,Наука и техника,Все,2011/07/27


80 процентов на дообучение, 20 процентов на валидацию

In [12]:
import numpy as np
import random
random.seed(1234)
np.random.seed(1234)
val_ind = random.sample(range(data1.shape[0]), 1000)
msk = np.random.rand(len(data1)) < 0.8
data1_train = data1[msk]
data1_test = data1[~msk]

In [13]:
print('------- TRAIN --------\n')
print(data1_train.count())

print('\n------- TEST -------\n')
print(data1_test.count())

print('\n------- ALL -------\n')
print(data1.count())

print('\n------- ALL/TEST -------\n')
print(data1_test.count() / data1.count() )

print('\n------- ALL/TRAIN -------\n')
print(data1_train.count() / data1.count())

------- TRAIN --------

url      12007
title    12007
text     12007
topic    12007
tags     12007
date     12007
dtype: int64

------- TEST -------

url      2993
title    2993
text     2993
topic    2993
tags     2993
date     2993
dtype: int64

------- ALL -------

url      15000
title    15000
text     15000
topic    15000
tags     15000
date     15000
dtype: int64

------- ALL/TEST -------

url      0.199533
title    0.199533
text     0.199533
topic    0.199533
tags     0.199533
date     0.199533
dtype: float64

------- ALL/TRAIN -------

url      0.800467
title    0.800467
text     0.800467
topic    0.800467
tags     0.800467
date     0.800467
dtype: float64


In [16]:
NAME_TRAIN_DATA = './nauka_train15.jsonl'
NAME_VALID_DATA = './nauka_valid15.jsonl'

PATH_TRAIN_DATA_SELF = 'nauka_train15.jsonl'
PATH_VALID_DATA_SELF = 'nauka_valid15.jsonl'

In [ ]:
Нужно всего два поля: summary, где будет эталон для суммаризации - заголовк; и text, где будет текст новостной статьи. 

In [17]:
import os
import sys
import json
import pathlib

with open(NAME_TRAIN_DATA, "w") as w, open(PATH_TRAIN_DATA_SELF, 'w', encoding="UTF-8") as ws:
    for row in data1_train.to_dict(orient="records"):
        r = {
             "summary": row["title"],
             "text": row["text"]
        }
        w.write(json.dumps(r, ensure_ascii=False) + "\n")
        ws.write(json.dumps(r, ensure_ascii=False) + "\n")

with open(NAME_VALID_DATA, "w") as w, open(PATH_VALID_DATA_SELF, 'w', encoding="UTF-8") as ws:
    for row in data1_test.to_dict(orient="records"):
        r = {
             "summary": row["title"],
             "text": row["text"]
        }
        w.write(json.dumps(r, ensure_ascii=False) + "\n")
        ws.write(json.dumps(r, ensure_ascii=False) + "\n")

if not pathlib.Path("models").exists():
  os.mkdir('models')

Сделаем так еще 5 раз и получим датасеты в нужном формате, разбитые на 12 jsonl файлов (на каждый из 6 датасетов по 2 файла: обучающая и валидационная выборки).

Займемся дообучением

Установим нужные библиотеки

In [1]:
!pip install transformers

Файл с нужными версиями чтобы дообучение запускалось

In [4]:
!python3 -m pip install -r /kaggle/input/summa-nach3/summ_epta/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.2/155.2 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.4/84.4 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 67.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 682.2/682.2 kB 42.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 764.0/764.0 kB 49.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 92.3 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 750.6/750.6 MB 1.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 92.4 MB/s eta 0:00:00:00:01

Снизу жесткий костыль, но без него не работает

In [5]:
#костыль
import torch
torch.cuda.empty_cache()
torch.cuda.is_available()
import os
!pip install pandas
!pip install lightning
! pip install -U git+https://github.com/huggingface/transformers.git
! pip install -U git+https://github.com/huggingface/accelerate.git
!python -m pip install --upgrade pip
!nvidia-smi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 811.0/811.0 kB 14.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.4/906.4 MB 1.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 57.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 75.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 23.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 10.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:0000:0100:01
     ━━━━━

Подключимся к hugging face, чтобы после дообучения сразу загружать туда модели

In [ ]:
from huggingface_hub import login
login()


In [7]:
from huggingface_hub import HfApi
api = HfApi()

Запустим дообучение. Сохранились логи дообучения модели на "спорте" на 15 тысяч статей, и модели совокупности "наука" + "спорт" на 15 тысяч статей.

In [7]:
!python3 /kaggle/input/summa-nach-200-2/summ_epta/train_model_data/ml_train_scripts/train.py \ # путь к трейн-файлу
  --config-file /kaggle/input/hugging-config2/gpt_training_config.json \ # путь к конфигурационному файлу
  --train-file /kaggle/input/data-15k/sport_train15.jsonl \ # путь к тренировочной выборке
  --val-file /kaggle/input/data-15k/sport_valid15.jsonl \ # путь к валидационной выборке
  --output-dir /kaggle/working/sport_15k # аутпут

/opt/conda/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/opt/conda/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
/opt/conda/lib/python3.10/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.2.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/p

После дообучения отправляем модель из kaggle сразу в hugging face репозиторий

In [8]:
api.upload_folder(
    folder_path="/kaggle/working/sport_15k",
    repo_id="Gnider/sport_6ep_15k"
)


rng_state.pth:   0%|          | 0.00/14.2k [00:00<?, ?B/s]

optimizer.pt:   0%|          | 0.00/1.00G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/501M [00:00<?, ?B/s]

scheduler.pt:   0%|          | 0.00/1.06k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.11k [00:00<?, ?B/s]

Upload 6 LFS files:   0%|          | 0/6 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/501M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Gnider/sport_6ep_15k/commit/c3e54ed095a11b7ca81ab21aa75169b04cfe22e8', commit_message='Upload folder using huggingface_hub', commit_description='', oid='c3e54ed095a11b7ca81ab21aa75169b04cfe22e8', pr_url=None, pr_revision=None, pr_num=None)

Очистим аутпут, чтобы освободить место для следующих моделей

In [9]:
!rm -rf /kaggle/working/*

Лог модели на миксе "наука" + "спорт"

In [10]:
!python3 /kaggle/input/summa-nach-200-2/summ_epta/train_model_data/ml_train_scripts/train.py \
  --config-file /kaggle/input/hugging-config2/gpt_training_config.json \
  --train-file /kaggle/input/data-15k/mix_train15.jsonl \
  --val-file /kaggle/input/data-15k/mix_valid15.jsonl \
  --output-dir /kaggle/working/mix_15k

/opt/conda/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/opt/conda/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
/opt/conda/lib/python3.10/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.2.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/p

Так же сохраняем

In [11]:
api.upload_folder(
    folder_path="/kaggle/working/mix_15k",
    repo_id="Gnider/mix_6ep_15k"
)


model.safetensors:   0%|          | 0.00/501M [00:00<?, ?B/s]

optimizer.pt:   0%|          | 0.00/1.00G [00:00<?, ?B/s]

rng_state.pth:   0%|          | 0.00/14.2k [00:00<?, ?B/s]

Upload 6 LFS files:   0%|          | 0/6 [00:00<?, ?it/s]

scheduler.pt:   0%|          | 0.00/1.06k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.11k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/501M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Gnider/mix_6ep_15k/commit/07d978e56298106a17d4f6578ec58cd1f9ed75aa', commit_message='Upload folder using huggingface_hub', commit_description='', oid='07d978e56298106a17d4f6578ec58cd1f9ed75aa', pr_url=None, pr_revision=None, pr_num=None)

# Итого было дообучено 6 моделей. Ссылки на сохраненные модели на Hugging Face:

### Обученные на "слабой" ruGPT-3 Small (125 млн параметров) на "больших" датасетах по 15,000 статей:

1) Наука: https://huggingface.co/Gnider/nauka_6ep_15k
2) Спорт: https://huggingface.co/Gnider/sport_6ep_15k
3) Наука + Спорт: https://huggingface.co/Gnider/mix_6ep_15k

### Обученные на "сильной" ruGPT-3 Medium (350 млн параметров) на "маленьких" датасетах по 6,900 статей:

1) Наука: https://huggingface.co/Gnider/nauka_6900_6ep_17_600_rugptmedium
2) Спорт: https://huggingface.co/Gnider/sport_6900_6ep_17_600_rugpt3medium
3) Наука + Спорт: https://huggingface.co/Gnider/mix_6900_6ep_17_600_tugpt3medium

# Теперь протестируем наши модели метрикой ROUGE

## "Слабая" модель Small, датасет 15к - Наука

подгрузим модель из нашей репы HF

In [98]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "Gnider/nauka_6ep_15k"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
model.to("cuda")

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50264, 768)
    (wpe): Embedding(2048, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50264, bias=False)
)

Возьмем валидационный файл и переведем его в csv (если вдруг забыли сохранить оригинальный csv датасета)

In [99]:
import pandas as pd
df = pd.read_json (r'/kaggle/input/data-15k/nauka_valid15.jsonl', lines = True)
df.to_csv (r'/kaggle/working/nauka_valid15.csv', index = None)

In [100]:
val = pd.read_csv('/kaggle/working/nauka_valid15.csv', encoding="utf-8")

Добавим колонку для предиктов

In [101]:
val['predicted'] = pd.Series(dtype='str')

In [102]:
val

,summary,text,predicted
0,Китай включился в атомную подводную гонку,Китай ведет интенсивные работы по обновлению ф...,NaN
1,Эксперимент по поиску гравитационных волн дал ...,Эксперимент по поиску гравитационных волн LIGO...,NaN
2,Из Apple iTunes скачали 300 миллионов учебных ...,"Из iTunes U, образовательного раздела магазина...",NaN
3,На искусственной коже впервые вырастили волосы,Японским ученым впервые удалось вырастить иску...,NaN
4,Россия в 2012 году получит десять бомбардировщ...,Министерство обороны России в 2012 году получи...,NaN
...,...,...,...
2988,ВМС США назовут первый авианосец морской пехот...,"Первый десантный корабль проекта LHA-6, которы...",NaN
2989,Apple обвинила Motorola в нарушении еще 12 пат...,Корпорация Apple обвинила компанию Motorola в ...,NaN
2990,Минкомсвязи позволит операторам выбрать станда...,Конкурсы на распределение частот для сетей свя...,NaN
2991,Ученые теоретически обосновали существование г...,Ученые из Гарвард-Смитсоновского астрофизическ...,NaN


Обозначим функцию, чтобы применять ее к текстам и делать предикты

In [103]:
def pred (x):
  article_text = x
  text_tokens = tokenizer(
    article_text,
    max_length=600,
    add_special_tokens=False,
    padding=False,
    truncation=True
   )["input_ids"]
  input_ids = text_tokens + [tokenizer.sep_token_id]
  input_ids = torch.LongTensor([input_ids]).to("cuda")
  output_ids = model.generate(
    input_ids=input_ids,
    no_repeat_ngram_size=0
  )
  summary = tokenizer.decode(output_ids[0], skip_special_tokens=False)
  summary = summary.split(tokenizer.sep_token)[1]
  summary = summary.split(tokenizer.eos_token)[0]
  print(summary)
  return(summary)

In [9]:
import torch

С помощью простого цикла прогоним каждый текст через модель, сохраим его созданную пустую колонку, выведем счетчик, чтобы понимать, как долго еще ждать, и прибавим счетчик

In [104]:
for i, row  in val.iterrows():
  z = pred(val.iat[i,1])
  val['predicted'][i] = z
  print(i)
  i=i+1

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


Китай ведет интенсивные работы по обновлению флота подводных лодок
0
Ученые нашли гравитационные волны
1
В iTunes U скачали более 300 миллионов учебных материалов
2
Ученые вырастили искусственную кожу с живыми волосяными луковицами
3
Минобороны получит десять фронтовых бомбардировщиков Су-34
4
НАСА увеличит расходы на космос в 2008 году
5
Рэй Долби умер в Нью-Йорке
6
Новая игра BioWare может стать перезапуском серии Command & Conquer
7
Square Enix раскрыла причины финансового неуспеха в 2012 году
8
Игра The Chronicles of Riddick: Assault on Dark Athena выйдет в апреле
9
Российские ядерные боеголовки оказались смертельно опасными
10
Apple отказалась от патентной тяжбы с Kodak
11
Новая Twisted Metal выйдет 4 октября
12
Физики предложили моделировать черные дыры с помощью мыльных пузырей
13
Названа дата назначения Болдена на новую должность
14
Консоль GameStick на Android профинансировали на Android
15
Создатели игры в StarCraft II учат искусственный интеллект игре в StarCraft II
16
Астро

In [105]:
val.head(20)

,summary,text,predicted
0,Китай включился в атомную подводную гонку,Китай ведет интенсивные работы по обновлению ф...,Китай ведет интенсивные работы по обновлению ф...
1,Эксперимент по поиску гравитационных волн дал ...,Эксперимент по поиску гравитационных волн LIGO...,Ученые нашли гравитационные волны
2,Из Apple iTunes скачали 300 миллионов учебных ...,"Из iTunes U, образовательного раздела магазина...",В iTunes U скачали более 300 миллионов учебных...
3,На искусственной коже впервые вырастили волосы,Японским ученым впервые удалось вырастить иску...,Ученые вырастили искусственную кожу с живыми в...
4,Россия в 2012 году получит десять бомбардировщ...,Министерство обороны России в 2012 году получи...,Минобороны получит десять фронтовых бомбардиро...
5,Бюджет NASA в 2008 году увеличится на полмилли...,Бюджет национального аэрокосмического агентств...,НАСА увеличит расходы на космос в 2008 году
6,Умер создатель технологии объемного звука,"В США 12 сентября 2013 года умер Рэй Долби, ра...",Рэй Долби умер в Нью-Йорке
7,BioWare приписали разработку новой Command & C...,Новой игрой BioWare может стать перезапуск сер...,Новая игра BioWare может стать перезапуском се...
8,Виновницей рекордных убытков Square Enix оказа...,Издательство Square Enix раскрыло причины свое...,Square Enix раскрыла причины финансового неусп...
9,Atari назвала дату выхода римейка The Chronicl...,Издательство Atari назвало окончательную дату ...,Игра The Chronicles of Riddick: Assault on Dar...


Теперь проведем анализ через ROUGE, чтобы понять, насколько наши заголовки похожи на эталон.

In [13]:
!pip install rouge

In [14]:
from rouge import Rouge

Обозначим эталон и предикт

In [106]:
model_out = val['predicted'].to_list()
reference = val['summary'].to_list()

In [107]:
rouge = Rouge()

In [108]:
rg = rouge.get_scores(model_out, reference, avg=True)
rg

{'rouge-1': {'r': 0.29829268162879924,
  'p': 0.2926637825101573,
  'f': 0.28961361845082423},
 'rouge-2': {'r': 0.14563154204818085,
  'p': 0.14416301496660053,
  'f': 0.14164840650755833},
 'rouge-l': {'r': 0.29077144502303237,
  'p': 0.2853826165587607,
  'f': 0.28233720325087963}}

Найдем среднее F-мер, чтобы получить усредненное качество генерации заголовков

In [109]:
f_scores = [metric['f'] for metric in rg.values()]
f_mean = sum(f_scores) / len(f_scores)

print(f"Среднее F-мер Small - 15k - Наука : {f_mean}")

Среднее F-мер Small - 15k - Наука : 0.23786640940308743


То есть получается, что "слабая" модель ruGPT-3 Small, дообученная на большом датасете в 15к статей, генерирует заголовки, которые в среднем похожи на оригинальный на 23,8 процентов (чисто формально).

Проделаем все то же самое со всеми остальными моделями

## "Слабая" модель Small, датасет 15к - Спорт

In [31]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "Gnider/sport_6ep_15k"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
model.to("cuda")

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50264, 768)
    (wpe): Embedding(2048, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50264, bias=False)
)

In [32]:
import pandas as pd
df = pd.read_json (r'/kaggle/input/data-15k/sport_valid15.jsonl', lines = True)
df.to_csv (r'/kaggle/working/sport_valid15.csv', index = None)

In [33]:
val = pd.read_csv('/kaggle/working/sport_valid15.csv', encoding="utf-8")

In [34]:
val['predicted'] = pd.Series(dtype='str')

In [35]:
val

,summary,text,predicted
0,Мамиашвили назвал ничтожествами взявших серебр...,Президент Федерации спортивной борьбы России (...,NaN
1,Загитова нашла себе соперниц,Олимпийская чемпионка по фигурному катанию Али...,NaN
2,Бывший игрок сборной России обесценил победу Ч...,Бывший игрок сборной России Андрей Канчельскис...,NaN
3,ФИФА заподозрила футболистов сборной России на...,"Футболисты сборной России, принимавшие участие...",NaN
4,Кириленко остался единственным кандидатом на п...,Исполнительный директор Российской федерации б...,NaN
...,...,...,...
2988,"Футболисты ""Барселоны"" поссорились во время игры","Футболисты ""Барселоны"" Херард Пике и Даниэл Ал...",NaN
2989,Тренер сборной Испании исключил свою команду и...,Главный тренер сборной Испании по футболу Луис...,NaN
2990,Работавший в России украинский тренер купил ап...,Бывший футболист сборной Украины Андрей Гусин ...,NaN
2991,"Соперник ""Рубина"" усилился бывшим вратарем ""Ма...","Греческий футбольный клуб ""Олимпиакос"", которы...",NaN


In [36]:
def pred (x):
  article_text = x
  text_tokens = tokenizer(
    article_text,
    max_length=600,
    add_special_tokens=False,
    padding=False,
    truncation=True
   )["input_ids"]
  input_ids = text_tokens + [tokenizer.sep_token_id]
  input_ids = torch.LongTensor([input_ids]).to("cuda")
  output_ids = model.generate(
    input_ids=input_ids,
    no_repeat_ngram_size=0
  )
  summary = tokenizer.decode(output_ids[0], skip_special_tokens=False)
  summary = summary.split(tokenizer.sep_token)[1]
  summary = summary.split(tokenizer.eos_token)[0]
  print(summary)
  return(summary)

In [37]:
import torch

In [39]:
for i, row  in val.iterrows():
  val['predicted'][i] = pred(val.iat[i,1])
  print(i)
  i=i+1

Мамиашвили обвинил россиянок в безволии
0
Олимпийская чемпионка по фигурному катанию Загитова высказалась о своих соперницах
1
Канчельскис раскритиковал главного тренера сборной России
2
Футболисты сборной России по футболу стали объектами расследования ФИФА
3
Дмитрий Домани снялся с выборов президента РФБ
4
Дэвид Пратт установил рекорд по скорости удаления с поля
5
Нэш пропустит чемпионат мира по хоккею
6
Суд Испании предъявит обвинения Неймару по делу о трансфере бразильского футболиста
7
Следствие возбудило уголовное дело по делу о смерти болельщика ЦСКА
8
«Стань человеком!» — проект, цель которого изменить отношение людей к фитнесу
9
Бразильский баскетболист побил рекорд результативности
10
"Сан-Антонио" выиграл пятый матч подряд
11
Кузнецова проиграла испанке в четвертьфинале итогового турнира WTA
12
"Арсенал" заинтересовался Коке
13
Валерий Газзаев сменит на пост президента "Динамо"
14
ЦСКА потребовал от Головина больше денег
15
«Реал» решил расстаться с Роналду за 200 миллионов 

In [40]:
val.head(20)

,summary,text,predicted
0,Мамиашвили назвал ничтожествами взявших серебр...,Президент Федерации спортивной борьбы России (...,Мамиашвили обвинил россиянок в безволии
1,Загитова нашла себе соперниц,Олимпийская чемпионка по фигурному катанию Али...,Олимпийская чемпионка по фигурному катанию Заг...
2,Бывший игрок сборной России обесценил победу Ч...,Бывший игрок сборной России Андрей Канчельскис...,Канчельскис раскритиковал главного тренера сбо...
3,ФИФА заподозрила футболистов сборной России на...,"Футболисты сборной России, принимавшие участие...",Футболисты сборной России по футболу стали объ...
4,Кириленко остался единственным кандидатом на п...,Исполнительный директор Российской федерации б...,Дмитрий Домани снялся с выборов президента РФБ
5,Футболист получил красную карточку на третьей ...,"Нападающий английского футбольного клуба ""Чипп...",Дэвид Пратт установил рекорд по скорости удале...
6,Лучший игрок хоккейного ЧМ-2007 пропустит ЧМ-2010,"Канадский форвард Рик Нэш, признанный лучшим и...",Нэш пропустит чемпионат мира по хоккею
7,Президенту «Барселоны» предъявят обвинение по ...,Верховный суд Испании предъявит обвинение през...,Суд Испании предъявит обвинения Неймару по дел...
8,СКП расследует дело о гибели болельщика ЦСКА,В следственный отдел по городу Раменское Следс...,Следствие возбудило уголовное дело по делу о с...
9,В Москве пройдет заключительный этап забега «С...,Заключительная в сезоне гонка «Стань человеком...,"«Стань человеком!» — проект, цель которого изм..."


In [41]:
model_out = val['predicted'].to_list()
reference = val['summary'].to_list()

In [42]:
rouge = Rouge()

In [43]:
rg = rouge.get_scores(model_out, reference, avg=True)
rg

{'rouge-1': {'r': 0.3949524238615434,
  'p': 0.3916492737732489,
  'f': 0.38597574127833884},
 'rouge-2': {'r': 0.2288138690577716,
  'p': 0.22780483104154967,
  'f': 0.22370644596976072},
 'rouge-l': {'r': 0.38312598416173294,
  'p': 0.38023307414705887,
  'f': 0.37458451678609883}}

In [44]:
f_scores = [metric['f'] for metric in rg.values()]
f_mean = sum(f_scores) / len(f_scores)

print(f"Среднее F-мер Small - 15k - Спорт: {f_mean}")

Среднее F-мер Small - 15k - Спорт: 0.3280889013447328


## "Слабая" модель Small, датасет 15к - Наука + Спорт

In [45]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "Gnider/mix_6ep_15k"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
model.to("cuda")

tokenizer_config.json:   0%|          | 0.00/1.50k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.61M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.74M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/826 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/501M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50264, 768)
    (wpe): Embedding(2048, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50264, bias=False)
)

In [46]:
import pandas as pd
df = pd.read_json (r'/kaggle/input/data-15k/mix_valid15.jsonl', lines = True)
df.to_csv (r'/kaggle/working/mix_valid15.csv', index = None)

In [47]:
val = pd.read_csv('/kaggle/working/mix_valid15.csv', encoding="utf-8")

In [48]:
val['predicted'] = pd.Series(dtype='str')

In [49]:
val

,summary,text,predicted
0,Четырехъядерные процессоры выпустят раньше вре...,Компания Intel уведомила своих партнеров об ув...,NaN
1,Российский центр разработок Nokia откроется в ...,Российский центр разработок Nokia откроется в ...,NaN
2,Планету Х обвинили в разрушении Солнечной системы,Астрономы из Испании и Великобритании обвинили...,NaN
3,Рефери боя между Ковалевым и Уордом поразился ...,"Рефери Роберт Берд, обслуживавший бой за титул...",NaN
4,Названа дата выхода The Beatles: Rock Band в Р...,Издательство Electronic Arts объявило дату вых...,NaN
...,...,...,...
2988,"Государство поможет ""МиГу"" рассчитаться с долгами",Премьер-министр России Владимир Путин подписал...,NaN
2989,Финляндия заказала у США тактические ракеты AT...,Финляндия запросила у США поставку 70 управляе...,NaN
2990,Фристайлистка Комиссарова вернулась в Москву п...,"Фристайлистка Мария Комиссарова, получившая пе...",NaN
2991,"Фанат ""Арсенала"" повесился после поражения от ...","Болельщик лондонского ""Арсенала"" кениец Сулейм...",NaN


In [50]:
def pred (x):
  article_text = x
  text_tokens = tokenizer(
    article_text,
    max_length=600,
    add_special_tokens=False,
    padding=False,
    truncation=True
   )["input_ids"]
  input_ids = text_tokens + [tokenizer.sep_token_id]
  input_ids = torch.LongTensor([input_ids]).to("cuda")
  output_ids = model.generate(
    input_ids=input_ids,
    no_repeat_ngram_size=0
  )
  summary = tokenizer.decode(output_ids[0], skip_special_tokens=False)
  summary = summary.split(tokenizer.sep_token)[1]
  summary = summary.split(tokenizer.eos_token)[0]
  print(summary)
  return(summary)

In [51]:
import torch

In [52]:
for i, row  in val.iterrows():
  val['predicted'][i] = pred(val.iat[i,1])
  print(i)
  i=i+1

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


Intel уведомила своих партнеров об увеличении темпов разработки и производства четырехъядерных процессоров
0
Nokia откроет "Сколково" в 2011 году
1
Астрономы обвинили Планету Х в разрушении Солнечной системы
2
Берд удивился судейским решениям после боя за титул чемпиона мира в тяжелом весе
3
Издательство Electronic Arts объявило дату выхода The Beatles: Rock Band на территории России
4
Lenovo и Toshiba анонсировали свои планшеты
5
Евгений Тищенко вышел в финал ОИ-2016
6
Впервые за полтора десятилетия эксплуатации парк ударных самолетов-невидимок F-117 достиг 100-процентной боевой готовности
7
Сотрудников «Боруссии» эвакуировали из-за найденного конверта с белым порошком
8
Тренер сборной Германии по хоккею отказался думать о матче с россиянами
9
Бразильская полиция арестовала футболиста Неймара
10
Спортивный арбитражный суд увеличил срок дисквалификации российских легкоатлеток
11
Илон Маск сравнил колонизацию Марса с открытием Америки англичанами
12
Астрономы обнаружили гигантского штор

In [53]:
val.head(20)

,summary,text,predicted
0,Четырехъядерные процессоры выпустят раньше вре...,Компания Intel уведомила своих партнеров об ув...,Intel уведомила своих партнеров об увеличении ...
1,Российский центр разработок Nokia откроется в ...,Российский центр разработок Nokia откроется в ...,"Nokia откроет ""Сколково"" в 2011 году"
2,Планету Х обвинили в разрушении Солнечной системы,Астрономы из Испании и Великобритании обвинили...,Астрономы обвинили Планету Х в разрушении Солн...
3,Рефери боя между Ковалевым и Уордом поразился ...,"Рефери Роберт Берд, обслуживавший бой за титул...",Берд удивился судейским решениям после боя за ...
4,Названа дата выхода The Beatles: Rock Band в Р...,Издательство Electronic Arts объявило дату вых...,Издательство Electronic Arts объявило дату вых...
5,Lenovo и Toshiba анонсировали новые планшеты,Компании Lenovo и Toshiba анонсировали собстве...,Lenovo и Toshiba анонсировали свои планшеты
6,Российский боксер Тищенко вышел в финал Олимпи...,"Российский боксер Евгений Тищенко, выступающий...",Евгений Тищенко вышел в финал ОИ-2016
7,F-117 достигли полной боевой готовности впервы...,Впервые за четверть века эксплуатации парк уда...,Впервые за полтора десятилетия эксплуатации па...
8,Офис дортмундской «Боруссии» эвакуировали из-з...,Сотрудников офиса немецкого футбольного клуба ...,Сотрудников «Боруссии» эвакуировали из-за найд...
9,Тренер немецких хоккеистов отказался думать о ...,Главный тренер сборной Германии по хоккею Марк...,Тренер сборной Германии по хоккею отказался ду...


In [54]:
model_out = val['predicted'].to_list()
reference = val['summary'].to_list()

In [55]:
rouge = Rouge()

In [56]:
rg = rouge.get_scores(model_out, reference, avg=True)
rg

{'rouge-1': {'r': 0.34009847393576076,
  'p': 0.33345615785636995,
  'f': 0.32985396548678325},
 'rouge-2': {'r': 0.18181551807512392,
  'p': 0.18020971626025545,
  'f': 0.17695913832938434},
 'rouge-l': {'r': 0.3295054566771905,
  'p': 0.32312314097621386,
  'f': 0.31958698161823773}}

In [57]:
f_scores = [metric['f'] for metric in rg.values()]
f_mean = sum(f_scores) / len(f_scores)

print(f"Среднее F-мер Small - 15k - Наука+Спорт: {f_mean}")

Среднее F-мер Small - 15k - Наука+Спорт: 0.27546669514480177


## "Сильная" модель Medium, датасет 6900 - Наука

In [58]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "Gnider/nauka_6900_6ep_17_600_rugptmedium"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
model.to("cuda")

tokenizer_config.json:   0%|          | 0.00/806 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.61M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.74M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/118 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.06k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 1024)
    (wpe): Embedding(2048, 1024)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-23): 24 x GPT2Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1024, out_features=50257, bias=False)
)

In [59]:
import pandas as pd
df = pd.read_json (r'/kaggle/input/datasety-fin/nauka_valid.jsonl', lines = True)
df.to_csv (r'/kaggle/working/nauka_valid.csv', index = None)

In [60]:
val = pd.read_csv('/kaggle/working/nauka_valid.csv', encoding="utf-8")

In [61]:
val['predicted'] = pd.Series(dtype='str')

In [62]:
val

,summary,text,predicted
0,На Земле нашли следы смертоносной космической ...,"Ученые из США и Бразилии выдвинули гипотезу, с...",NaN
1,Предсказаны масштабные разрушения в северных с...,"Международная группа ученых из Финляндии, Норв...",NaN
2,Созданы вечно текущие капли первородной материи,Физики Брукхейвенской национальной лаборатории...,NaN
3,Объяснена смертельная опасность соли,Японские ученые Национального института фундам...,NaN
4,Найдена неизвестная причина гибели клеток у людей,Ученые Юго-Западного медицинского центра Униве...,NaN
...,...,...,...
1382,Астрофизики измерили скорость вращения черной ...,"Астрофизики вычислили, что черная дыра в центр...",NaN
1383,Мозг оказался способен различать согласные за ...,"Нейробиологи установили, что мозг человека спо...",NaN
1384,Орнитологам понадобился робот-воробей,Орнитологи из университета Дьюка в Северной Ка...,NaN
1385,Геологи обнаружили скрытый континент в Индийск...,Геологи обнаружили в районе Маврикия следы дре...,NaN


In [63]:
def pred (x):
  article_text = x
  text_tokens = tokenizer(
    article_text,
    max_length=600,
    add_special_tokens=False,
    padding=False,
    truncation=True
   )["input_ids"]
  input_ids = text_tokens + [tokenizer.sep_token_id]
  input_ids = torch.LongTensor([input_ids]).to("cuda")
  output_ids = model.generate(
    input_ids=input_ids,
    no_repeat_ngram_size=0
  )
  summary = tokenizer.decode(output_ids[0], skip_special_tokens=False)
  summary = summary.split(tokenizer.sep_token)[1]
  summary = summary.split(tokenizer.eos_token)[0]
  print(summary)
  return(summary)

In [64]:
import torch

In [65]:
for i, row  in val.iterrows():
  val['predicted'][i] = pred(val.iat[i,1])
  print(i)
  i=i+1

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


Объяснено массовое вымирание животных в конце плиоцена
0
Ученые пришли к выводу о неизбежном разрушении инфраструктуры в северных полярных регионах
1
Физики впервые получили капли кварк-глюонной плазмы
2
Ученые раскрыли механизм развития гипертонии
3
Ученые раскрыли секрет иммунитета
4
Гравитационные волны опровергли Стандартную модель
5
Землетрясение в Новой Зеландии сблизило Южный и Северный острова
6
В Гонконге зафиксировали массовое заражение вирусом гепатита Е
7
Ученые нашли способ борьбы с глобальным потеплением
8
Ученые объяснили эффективность ингибиторов PD-1
9
Названа главная функция перинейрональных сетей
10
Раскрыт самый древний некрополь Древнего Царства
11
Ученые нашли причину глобального потепления
12
Ученые раскрыли молекулярный механизм рака
13
В США смыт Восточный остров Гавайского архипелага
14
Названа главная причина катастрофического извержения супервулкана Таупо
15
Арктический лед исчез из-за глобального потепления
16
Врачи предупредили о росте заболеваемости остры

In [66]:
val.head(20)

,summary,text,predicted
0,На Земле нашли следы смертоносной космической ...,"Ученые из США и Бразилии выдвинули гипотезу, с...",Объяснено массовое вымирание животных в конце ...
1,Предсказаны масштабные разрушения в северных с...,"Международная группа ученых из Финляндии, Норв...",Ученые пришли к выводу о неизбежном разрушении...
2,Созданы вечно текущие капли первородной материи,Физики Брукхейвенской национальной лаборатории...,Физики впервые получили капли кварк-глюонной п...
3,Объяснена смертельная опасность соли,Японские ученые Национального института фундам...,Ученые раскрыли механизм развития гипертонии
4,Найдена неизвестная причина гибели клеток у людей,Ученые Юго-Западного медицинского центра Униве...,Ученые раскрыли секрет иммунитета
5,Подтверждена теория Эйнштейна в ущерб Теории в...,Группа ученых из коллаборации LIGO и Virgo не ...,Гравитационные волны опровергли Стандартную мо...
6,Замечен сдвиг суши из-за крупной катастрофы,Ученые новозеландского Королевского научно-исс...,Землетрясение в Новой Зеландии сблизило Южный ...
7,Зафиксирована новая опасная болезнь,Ученые зафиксировали в Гонконге несколько случ...,В Гонконге зафиксировали массовое заражение ви...
8,Борьбу с глобальным потеплением признали эконо...,Ученые Гарвардского университета пришли к выво...,Ученые нашли способ борьбы с глобальным потепл...
9,Доказана неожиданная польза ожирения,"Ученые Калифорнийского университета объяснили,...",Ученые объяснили эффективность ингибиторов PD-1


In [67]:
model_out = val['predicted'].to_list()
reference = val['summary'].to_list()

In [68]:
rouge = Rouge()

In [69]:
rg = rouge.get_scores(model_out, reference, avg=True)
rg

{'rouge-1': {'r': 0.23053396775458768,
  'p': 0.22384348108032298,
  'f': 0.22178391429650465},
 'rouge-2': {'r': 0.09642041047953097,
  'p': 0.09429190521545187,
  'f': 0.0927840079774913},
 'rouge-l': {'r': 0.22199518618047798,
  'p': 0.21598933190497693,
  'f': 0.2137787614370897}}

In [70]:
f_scores = [metric['f'] for metric in rg.values()]
f_mean = sum(f_scores) / len(f_scores)

print(f"Среднее F-мер Medium - 6900 - Наука: {f_mean}")

Среднее F-мер Medium - 6900 - Наука: 0.17611556123702854


## "Сильная" модель Medium, датасет 6900 - Спорт

In [71]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "Gnider/sport_6900_6ep_17_600_rugpt3medium"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
model.to("cuda")

tokenizer_config.json:   0%|          | 0.00/806 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.61M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.74M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/118 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.06k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 1024)
    (wpe): Embedding(2048, 1024)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-23): 24 x GPT2Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1024, out_features=50257, bias=False)
)

In [72]:
import pandas as pd
df = pd.read_json (r'/kaggle/input/datasety-fin/sport_valid.jsonl', lines = True)
df.to_csv (r'/kaggle/working/sport_valid.csv', index = None)

In [73]:
val = pd.read_csv('/kaggle/working/sport_valid.csv', encoding="utf-8")

In [74]:
val['predicted'] = pd.Series(dtype='str')

In [75]:
val

,summary,text,predicted
0,МОК потребовал вернуть медали трех олимпийских...,Международный олимпийский комитет (МОК) потреб...,NaN
1,Моуринью сравнил футболистов с мебелью,Главный тренер «Манчестер Юнайтед» Жозе Моурин...,NaN
2,Россиянин не глядя отбил летевшую в пустые вор...,Вратарь «Тампа-Бэй Лайтнинг» Андрей Василевски...,NaN
3,В деле российских биатлонистов нашли след Родч...,Президент Союза биатлонистов России (СБР) Влад...,NaN
4,Российский биатлон отреагировал на обвинения а...,"В Союзе биатлонистов России (СБР) подтвердили,...",NaN
...,...,...,...
1350,Парагвайского футболиста уличили в сексуальной...,"Парагвайский футболист Хонатан Фаббро, выступа...",NaN
1351,Тренер сборной России прокомментировала переим...,Главный тренер сборной России Татьяна Покровск...,NaN
1352,Представлен официальный постер к бою Мэйуэзер ...,Телеканал Showtime представил официальный пост...,NaN
1353,На стадионе «Санкт-Петербург» протекла крыша,Во время матча второго тура Российской футболь...,NaN


In [76]:
def pred (x):
  article_text = x
  text_tokens = tokenizer(
    article_text,
    max_length=600,
    add_special_tokens=False,
    padding=False,
    truncation=True
   )["input_ids"]
  input_ids = text_tokens + [tokenizer.sep_token_id]
  input_ids = torch.LongTensor([input_ids]).to("cuda")
  output_ids = model.generate(
    input_ids=input_ids,
    no_repeat_ngram_size=0
  )
  summary = tokenizer.decode(output_ids[0], skip_special_tokens=False)
  summary = summary.split(tokenizer.sep_token)[1]
  summary = summary.split(tokenizer.eos_token)[0]
  print(summary)
  return(summary)

In [77]:
import torch

In [78]:
for i, row  in val.iterrows():
  val['predicted'][i] = pred(val.iat[i,1])
  print(i)
  i=i+1

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1375: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


Российские бобслеисты потребовали вернуть золотые медали Олимпиады в Сочи
0
Жозе Моуринью раскритиковал «Манчестер Юнайтед»
1
Вратарь «Тампа-Бэй Лайтнинг» совершил сэйв пяткой в матче регулярного чемпионата НХЛ
2
Российские биатлонисты оказались виновными в махинациях с допингом
3
Сборную России обвинили в нарушении антидопинговых правил
4
Российская биатлонистка Екатерина Юрлова-Перхт выиграла бронзовую медаль в спринте на этапе Кубка мира в австрийском Хохфильцене
5
«Барселона» не заинтересована в продаже Дембеле
6
Овечкин записал на свой счет хет-трик в игре против «Детройт Ред Уингс»
7
«Вест Бромвич» забил гол после сольного прохода в матче Кубка Англии
8
У Кокорина ухудшится здоровье из-за отсутствия тренировок
9
Олимпийский чемпион Йоханнес Бе рассказал об отношении к российскому биатлонисту Александру Логинову
10
Локомотив проиграл «Шальке» в выездном матче Лиги чемпионов
11
Валерия Газзаева освистали на турнире по борьбе
12
Марокканский полузащитник «Акхисар Беледиеспор» забил 

In [79]:
val.head(20)

,summary,text,predicted
0,МОК потребовал вернуть медали трех олимпийских...,Международный олимпийский комитет (МОК) потреб...,Российские бобслеисты потребовали вернуть золо...
1,Моуринью сравнил футболистов с мебелью,Главный тренер «Манчестер Юнайтед» Жозе Моурин...,Жозе Моуринью раскритиковал «Манчестер Юнайтед»
2,Россиянин не глядя отбил летевшую в пустые вор...,Вратарь «Тампа-Бэй Лайтнинг» Андрей Василевски...,Вратарь «Тампа-Бэй Лайтнинг» совершил сэйв пят...
3,В деле российских биатлонистов нашли след Родч...,Президент Союза биатлонистов России (СБР) Влад...,Российские биатлонисты оказались виновными в м...
4,Российский биатлон отреагировал на обвинения а...,"В Союзе биатлонистов России (СБР) подтвердили,...",Сборную России обвинили в нарушении антидопинг...
5,Российская биатлонистка впервые за два года за...,Российская биатлонистка Екатерина Юрлова-Перхт...,Российская биатлонистка Екатерина Юрлова-Перхт...
6,«Барселона» повесила на вечно опаздывающего иг...,«Барселона» не заинтересована в продаже напада...,«Барселона» не заинтересована в продаже Дембеле
7,Овечкин побил рекорд Павла Буре,Российский форвард «Вашингтон Кэпиталс» Алекса...,Овечкин записал на свой счет хет-трик в игре п...
8,16-летний футболист получил мяч в центре поля ...,Английский форвард «Вест Бромвич Альбион» Морг...,«Вест Бромвич» забил гол после сольного проход...
9,Кокорин столкнулся с перспективой стать калеко...,"У нападающего «Зенита» Александра Кокорина, на...",У Кокорина ухудшится здоровье из-за отсутствия...


In [80]:
model_out = val['predicted'].to_list()
reference = val['summary'].to_list()

In [81]:
rouge = Rouge()

In [82]:
rg = rouge.get_scores(model_out, reference, avg=True)
rg

{'rouge-1': {'r': 0.27511946233348444,
  'p': 0.2410568924298814,
  'f': 0.24841651096337078},
 'rouge-2': {'r': 0.12144846543370541,
  'p': 0.10521934880453969,
  'f': 0.10854700246492681},
 'rouge-l': {'r': 0.2653956211889792,
  'p': 0.23216715552088946,
  'f': 0.2394158781396819}}

In [83]:
f_scores = [metric['f'] for metric in rg.values()]
f_mean = sum(f_scores) / len(f_scores)

print(f"Среднее F-мер Medium - 6900 - Спорт: {f_mean}")

Среднее F-мер Medium - 6900 - Спорт: 0.19879313052265982


## "Сильная" модель Medium, датасет 6900 - Наука и спорт

In [84]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "Gnider/mix_6900_6ep_17_600_tugpt3medium"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
model.to("cuda")

tokenizer_config.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.61M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.74M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/105 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/1.06k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 1024)
    (wpe): Embedding(2048, 1024)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-23): 24 x GPT2Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=1024, out_features=50257, bias=False)
)

In [85]:
import pandas as pd
df = pd.read_json (r'/kaggle/input/datasety-fin/mix_valid.jsonl', lines = True)
df.to_csv (r'/kaggle/working/mix_valid.csv', index = None)

In [86]:
val = pd.read_csv('/kaggle/working/mix_valid.csv', encoding="utf-8")

In [87]:
val['predicted'] = pd.Series(dtype='str')

In [88]:
val

,summary,text,predicted
0,Фуркад пожелал россиянину лишения золота Олимп...,Пятикратный олимпийский чемпион Мартен Фуркад ...,NaN
1,Роналду предложили вечно считать лучшим футбол...,Главный тренер итальянского клуба «Наполи» Кар...,NaN
2,Два игрока сборной Хорватии обратились в больн...,Два футболиста сборной Хорватии были вынуждены...,NaN
3,Рогозин нашел мистический след в отмене пуска ...,Российский вице-премьер Дмитрий Рогозин заявил...,NaN
4,Российский биатлон обрел нового президента,Четырехкратный чемпион мира по биатлону Владим...,NaN
...,...,...,...
1382,Автору победного гола в ворота хорватов пообещ...,Автор победного гола в матче против сборной Хо...,NaN
1383,Роналду явился на тренировку с подбитым глазом,Португальский форвард туринского «Ювентуса» Кр...,NaN
1384,Раскрыты подробности о прошлом крупнейшего ост...,Две группы американских и британских ученых вы...,NaN
1385,Последствия ядерной войны показали на видео,"YouTube-канал Life Noggin представил видео, в ...",NaN


In [89]:
def pred (x):
  article_text = x
  text_tokens = tokenizer(
    article_text,
    max_length=600,
    add_special_tokens=False,
    padding=False,
    truncation=True
   )["input_ids"]
  input_ids = text_tokens + [tokenizer.sep_token_id]
  input_ids = torch.LongTensor([input_ids]).to("cuda")
  output_ids = model.generate(
    input_ids=input_ids,
    no_repeat_ngram_size=0
  )
  summary = tokenizer.decode(output_ids[0], skip_special_tokens=False)
  summary = summary.split(tokenizer.sep_token)[1]
  summary = summary.split(tokenizer.eos_token)[0]
  print(summary)
  return(summary)

In [90]:
import torch

In [91]:
for i, row  in val.iterrows():
  val['predicted'][i] = pred(val.iat[i,1])
  print(i)
  i=i+1

Устюгова лишили золотой медали Олимпиады
0
Тренер «Ювентуса» раскритиковал Роналду
1
Хорватского футболиста наказали за крик «Слава Украине!»
2
Российского вице-премьера обвинили в мистическом происшествии
3
Драчев победил на выборах президента СБР
4
Кокорину посоветовали побить семью
5
Венгер покинет «Арсенал» по окончании сезона
6
Названы гонорары бойцов на UFC 136
7
Найдены следы крови на плащанице Иисуса Христа
8
«Реал» готов расстаться с Роналду
9
Ученые опровергли популярную теорию эволюции
10
Боец MMA нокаутировал соперника «летающим треугольником»
11
Черчесов объяснил назначение Дзюбы капитаном сборной
12
В МГУ нашли лекарство от паралича
13
Названа главная причина глобального голода
14
Найден новый эффективный способ борьбы с раком
15
Шевчук попал в центр внимания украинских СМИ
16
Исполнительный директор UFC попал в больницу из-за нападения Макгрегора
17
В кукурузе нашли движущую силу для роботов
18
ЦСКА и «Спартак» сыграли вничью в дерби
19
Найдена главная причина бессонницы

In [92]:
val.head(20)

,summary,text,predicted
0,Фуркад пожелал россиянину лишения золота Олимп...,Пятикратный олимпийский чемпион Мартен Фуркад ...,Устюгова лишили золотой медали Олимпиады
1,Роналду предложили вечно считать лучшим футбол...,Главный тренер итальянского клуба «Наполи» Кар...,Тренер «Ювентуса» раскритиковал Роналду
2,Два игрока сборной Хорватии обратились в больн...,Два футболиста сборной Хорватии были вынуждены...,Хорватского футболиста наказали за крик «Слава...
3,Рогозин нашел мистический след в отмене пуска ...,Российский вице-премьер Дмитрий Рогозин заявил...,Российского вице-премьера обвинили в мистическ...
4,Российский биатлон обрел нового президента,Четырехкратный чемпион мира по биатлону Владим...,Драчев победил на выборах президента СБР
5,Мать Кокорина объяснила улыбку сына при задерж...,Мать футболиста Александра Кокорина Светлана в...,Кокорину посоветовали побить семью
6,Эпоха Венгера в «Арсенале» подошла к концу,Главный тренер лондонского «Арсенала» Арсен Ве...,Венгер покинет «Арсенал» по окончании сезона
7,Названы гонорары победителей турнира UFC в России,Стали известны гонорары бойцов на турнире Абсо...,Названы гонорары бойцов на UFC 136
8,Туринская плащаница оказалась художественной п...,Специалисты из Университета Ливерпуля (Великоб...,Найдены следы крови на плащанице Иисуса Христа
9,«Реал» назвал цену Роналду,Руководство мадридского «Реала» озвучило сумму...,«Реал» готов расстаться с Роналду


In [93]:
model_out = val['predicted'].to_list()
reference = val['summary'].to_list()

In [94]:
rouge = Rouge()

In [95]:
rg = rouge.get_scores(model_out, reference, avg=True)
rg

{'rouge-1': {'r': 0.2648061323439838,
  'p': 0.2662836505540813,
  'f': 0.25969060765372476},
 'rouge-2': {'r': 0.12828191795387167,
  'p': 0.13041842592808994,
  'f': 0.1265761692933671},
 'rouge-l': {'r': 0.25871405491088273,
  'p': 0.2600423071822405,
  'f': 0.25367108893331947}}

In [96]:
f_scores = [metric['f'] for metric in rg.values()]
f_mean = sum(f_scores) / len(f_scores)

print(f"Среднее F-мер Medium - 6900 - Наука+Спорт: {f_mean}")

Среднее F-мер Medium - 6900 - Наука+Спорт: 0.21331262196013712
